# Automatic comparison of STIX and AIA images

The aim of this script is to take as input the FITS files with STIX images and then downloading, processing and doing cutout of AIA 1600 or AIA 1700 images


### Imports and functions

In [1]:
import astropy.units as u
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from aiapy.calibrate import fix_observer_location, normalize_exposure, register, update_pointing
from astropy.coordinates import SkyCoord
from astropy.time import Time

from sunpy.coordinates import Helioprojective
from sunpy.coordinates.ephemeris import get_earth
from sunpy.coordinates.frames import HeliocentricEarthEcliptic, HeliographicStonyhurst
from sunpy.map import Map, all_coordinates_from_map, make_fitswcs_header
from sunpy.net import attrs as a
from sunpy.net import Fido

In [2]:
plt.rcParams.update({'font.size': 14})

### Global parameters

In [3]:
# Folder containing the STIX maps
# folder_stix = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/'
folder_stix = '/net/galilei/work2/fschuller/data/STIX-maps/'
# working on sylvaner:
# folder_stix = '/work2/fschuller/STIX/STIX-maps/'

# Folder where to store the reprojected AIA maps
folder_store_rmaps = folder_stix+"AIA/"

# Folder where to store the AIA full-disk maps
folder_store = folder_stix+"AIA/"

# Folder where to store the overview plots
folder_plots = folder_stix

# Email associated with the JSOC account
# Not registered yet? Go to: http://jsoc.stanford.edu/ajax/register_email.html
email = 'fschuller@aip.de'

# Wavelengths of the AIA maps to download
# wavelength = 1600
wavelength = 1700

# FOV of the AIA maps
fov = [600, 600]

# Final FOV of the reprojected AIA maps with STIX
fov_plot = [360, 360]

# Path to the Frederic's table
# frederic_csv = '/home/afbattaglia/Software/STIX_estimate_pointing_error/Flare-location_test-table.csv'

### AIA related functions

In [4]:
def process_aia(map):
    """
    Process the AIA map to level 1.5 and
    update the pointing information in the header.
    """

    # Calibrate the map
    map = register(update_pointing(map))
    map = normalize_exposure(map)
    map = fix_observer_location(map)

    return map

In [51]:
def download_aia(peak_time, wavelength, email, prep = True, folder_store=''):
    """
    Download the AIA map
    """

    # JSOC email
    #jsoc_email = os.environ[email]

    # Define starting and ending time of the interval
    # for searching the AIA map (default 12 seconds)
    dt = 12
    time_start = Time(peak_time)-dt*u.s
    time_end = Time(peak_time)+dt*u.s

    # Search for the data
    query = Fido.search(a.Time(time_start, time_end),
                    a.Wavelength(wavelength*u.angstrom),
                    a.Sample(48*u.s),
                    a.jsoc.Series.aia_lev1_uv_24s,
                    a.jsoc.Notify(email),
                    a.jsoc.Segment.image
                    )
    
    # Make sure that we found at least one data product
    if len(query[0]) == 0:
        # otherwise, increase time range to query
        print("WARNING: no AIA data found within +/-12 s, increasing time range of query")
        time_start -= dt*u.s
        time_end += dt*u.s
        query = Fido.search(a.Time(time_start, time_end),
                    a.Wavelength(wavelength*u.angstrom),
                    a.Sample(48*u.s),
                    a.jsoc.Series.aia_lev1_uv_24s,
                    a.jsoc.Notify(email),
                    a.jsoc.Segment.image
                    )

    # Download the data
    file_downloaded = Fido.fetch(query[0][0], path=folder_store)

    if file_downloaded == []:
        print('No data found. We try to download the data from ROB now...')
        file_downloaded = Fido.fetch(query[0][0], site='ROB', path=folder_store)

    # it may be that the result contains more than one file, therefore:
    if len(file_downloaded) > 1:
        file_downloaded = file_downloaded[0]
    
    the_map = Map(file_downloaded)

    # If prep == True, then process the map
    if prep == True:
        the_map = process_aia(the_map)

    return the_map

In [6]:
def reproject_aia2so(aia_map, stix_map, out_shape = (1024, 1024)):
    """
    Reproject the AIA map to the Solar Orbiter view
    """

    # Get the coordinates of the center of the STIX map in arcsec
    all_coords = all_coordinates_from_map(stix_map)
    # get the coordinates of the center of the map
    center = SkyCoord(all_coords.Tx.mean(), all_coords.Ty.mean(), frame=stix_map.coordinate_frame)
    mapcenter = [center.Tx.value, center.Ty.value]

    # Define the reference coordinate
    ref_coord = SkyCoord(
                     mapcenter[0]*u.arcsec, mapcenter[1]*u.arcsec,
                     #0*u.arcsec, 0*u.arcsec,
                     frame='helioprojective', 
                     obstime=stix_map.date, 
                     observer=stix_map.observer_coordinate)


    # Reference pixel in the AIA map
    #ref_pix = stix_map.world_to_pixel(center)

    # Distance AIA and SO to the Sun
    dist_aia = aia_map.observer_coordinate.radius.value
    dist_so  = stix_map.observer_coordinate.radius.value
    factor = dist_aia / dist_so
    pixel_size = [float(aia_map.scale[0]/u.arcsec*u.pixel) * factor, 
                  float(aia_map.scale[1]/u.arcsec*u.pixel) * factor]

    # Create a FITS WCS header for the reference coordinate and frame
    header = make_fitswcs_header(
        out_shape,
        ref_coord,
        scale = pixel_size*u.arcsec/u.pixel
        #reference_pixel = ref_pix*u.pix
        )

    # Reproject the AIA map
    map = aia_map.reproject_to(header)
    
    return map, mapcenter

In [7]:
def hgs2hpc(hsg_coord, date, observer):
    '''
    This function takes as an input the Heliographic Stonyhurts coordinates
    and returns the Helioprojective coordinates as observed by Earth.
     - hsg_coord must be a 2D array [lon (deg), lat (deg)]
     - date of the observation
     - observer must be the result of map.observer_coordinate (of an AIA map)
    '''

    # SkyCoord of the input coordinates
    hsg_skycoord = SkyCoord(hsg_coord[0]*u.deg, 
                            hsg_coord[1]*u.deg, 
                            #obstime=date,
                            frame=HeliographicStonyhurst(obstime=date))

    # Transform to Helioprojective cartesian
    hpc_coord = hsg_skycoord.transform_to(Helioprojective(obstime=date,
                                          observer=observer))

    return hpc_coord

In [8]:
def flare_coord_to_float(coord_string):
    # Convert the flare coordinates from string to 2 float values
    sign_ns = 1
    sign_ew = 1
    if coord_string[0] == 'S': sign_ns = -1
    if coord_string[3] == 'E': sign_ew = -1
    coord_ns = float(coord_string[1:3])
    coord_ew = float(coord_string[4:6])
    lon = (sign_ew * coord_ew) #* u.deg
    lat = (sign_ns * coord_ns) #* u.deg
    return (lon,lat)
    # print("Flare location: %3i , %3i"%(lon,lat))


In [9]:
def transform_HGS_to_HPC(coord, peak_time):
    # Transform the locations from HGS string to HPC coordinates
    # Get the Earth coordinates in hgs
    earth_hgs = get_earth(Time(peak_time).isot)

    # Transform the coordinates of the flare location from hgs to hpc
    hpc_coord = hgs2hpc([coord[0], coord[1]], peak_time, earth_hgs)

    # Store the coordinates
    flare_loc_earth_hpc = [hpc_coord.Tx.value,hpc_coord.Ty.value]
    # print("Helioprojective location: %7.1f , %7.1f"%(flare_loc_earth_hpc[0],flare_loc_earth_hpc[1]))
    
    return flare_loc_earth_hpc



In [63]:
def plot_aia_stix_overlay(uid, peak_time_so, start_time_so, end_time_so, 
                          energy_low, energy_high, flare_loc, wavelength=1700):
    # Open STIX map, download AIA map closest to peak time, re-project to STIX viewpoint,
    # and generate a plot showing the overlay of STIX on top
    # stix_map_file = glob.glob(folder_stix + '*'+str(uid)+'*.fits')
    # STIX maps computed without applying any pointing correction:
    stix_map_file = glob.glob(folder_stix + peak_time_so + '_' + str(uid) + '*No-SAS_mem.fits')
    print("Found STIX map: ",stix_map_file)
    stix_map = Map(stix_map_file[0])
    
    # Get the light travel time correction from the FITS header
    ltt = stix_map.meta['EAR_TDEL']
    print('Light travel time correction: '+str(ltt)+' s')

    # Time at Earth
    # time_earth = Time(peak_time_so) + ltt*u.s
    # Better use the average time used to generate STIX map:
    time_earth = Time(stix_map.meta['date-avg']) + ltt*u.s

    # Download the AIA map
    print("Downloading AIA image...")
    aia_map = download_aia(time_earth, wavelength, email, folder_store=folder_store)
    
    # modify the value for rsun in AIA map to match the one in STIX map
    aia_map.meta['rsun_ref'] = stix_map.meta['rsun_ref']

    # convert flare location to HPC coordinates
    coord = flare_coord_to_float(flare_loc)
    flare_loc_earth_hpc = transform_HGS_to_HPC(coord, peak_time_so)

    # Extract the submap according to the flare location as seen from Earth
    bottom_left = SkyCoord((flare_loc_earth_hpc[0]-fov[0]/2)*u.arcsec,
                       (flare_loc_earth_hpc[1]-fov[1]/2)*u.arcsec,
                       frame=aia_map.coordinate_frame)
    top_right = SkyCoord((flare_loc_earth_hpc[0]+fov[0]/2)*u.arcsec,
                     (flare_loc_earth_hpc[1]+fov[1]/2)*u.arcsec,
                     frame=aia_map.coordinate_frame)
    submap = aia_map.submap(bottom_left, top_right=top_right)
    
    # Reproject the AIA map to the STIX view
    rmap_aia, stix_center = reproject_aia2so(submap, stix_map)

    # Save the reprojected map
    rmap_aia.save(folder_store_rmaps+'reprojected-AIA'+str(wavelength)+'_'+str(peak_time_so)+'.fits', overwrite=1)

    # Select sub-maps covering the same region with both instruments
    # but shift center by 30'' upwards, to take into account at least
    # some fraction of the systematic error
    bottom_left_plot = SkyCoord((stix_center[0]-fov_plot[0]/2)*u.arcsec,
                            (stix_center[1]-fov_plot[1]/2 + 30.)*u.arcsec,
                            frame=rmap_aia.coordinate_frame)
    top_right_plot = SkyCoord((stix_center[0]+fov_plot[0]/2)*u.arcsec,
                          (stix_center[1]+fov_plot[1]/2 + 30.)*u.arcsec,
                          frame=rmap_aia.coordinate_frame)
    sub_rmap_aia = rmap_aia.submap(bottom_left_plot, top_right=top_right_plot)
    sub_stix_map = stix_map.submap(bottom_left_plot, top_right=top_right_plot)

    ##########
    # Here comes the plot
    fig = plt.figure(figsize=(10,10), dpi=100, facecolor='white')
    ax = plt.subplot(projection=sub_rmap_aia)
    sub_rmap_aia.plot(axes=ax)
    sub_stix_map.draw_contours([30,50,70,90]*u.pct, axes=ax, colors='black')
    lon = ax.coords[0]
    lon.display_minor_ticks(True)
    lat = ax.coords[1]
    lat.display_minor_ticks(True)
    lon.grid(color='black', alpha=1, linestyle='solid')
    lat.grid(color='black', alpha=1, linestyle='solid')
    ax.set_title(aia_map.meta['date-obs']+'   Earth UT')


    # Add some text caption
    x_txt = 0.04
    y_txt = 0.96
    dy = 0.03

    ax.text(x_txt, y_txt, 'Data request UID: '+str(uid),
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='black', fontsize=15)
    ax.text(x_txt, y_txt-dy, 'Time range (SO UT): '+start_time_so+' - '+end_time_so,
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='black', fontsize=15)
    ax.text(x_txt, y_txt-2*dy, 'Energy range (keV): %2s - %2s'%(str(energy_low),str(energy_high)),
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='black', fontsize=15)

    # Export to PNG
    plt.savefig(folder_plots+'AIA'+str(wavelength)+'-STIX_'+str(peak_time_so)+'_'+str(uid)+'.png')
    
    # Close figure to free memory (?)
    plt.close()


## Loop over many (all) events

In [71]:
# Read file with all parameters
input_csv = '/home/fschuller/Documents/SolarOrb/Flare_location_SDO+STIX-imaging.csv'
df = pd.read_csv(input_csv, header=1)

df


,Peak time (UTC),GOES,SSW loc.,UID,t min,t max,E min,E max
0,2021-03-02T23:56,C1.2,S19E43,2103020009,2021-03-02T23:56:40,2021-03-02T23:58:10,4,15
1,2021-05-07T18:50,M3.9,N17E78,2105070034,2021-05-07T18:50:00,2021-05-07T19:09:00,15,28
2,2021-07-24T00:25,C4.3,S16E52,2107240003,2021-07-24T00:24:25,2021-07-24T00:27:30,10,25
3,2021-08-13T20:59,C1.4,N24E00,2108130003,2021-08-13T21:03:55,2021-08-13T21:05:55,4,20
4,2021-08-26T23:16,C3.9,S30E21,2108260029,2021-08-26T23:17:55,2021-08-26T23:19:35,20,28
...,...,...,...,...,...,...,...,...
67,2023-02-24T20:23,M3.7,N29W24,2302247218,2023-02-24T20:16:35,2023-02-24T20:18:50,25,40
68,2023-02-25T19:34,M6.4,N24W45,2302250933,2023-02-25T19:21:50,2023-02-25T19:25:40,25,40
69,2023-02-28T17:47,M8.6,N27W29,2302282620,2023-02-28T17:40:45,2023-02-28T17:41:17,25,50
70,2023-03-05T21:32,M5.0,N18W60,2303057087,2023-03-05T21:28:40,2023-03-05T21:33:15,25,50


In [72]:
# Store variables of interests
peak_times_so = list(df.iloc[:,0])
flare_loc_earth_string = list(df.iloc[:,2])
uid = list(df.iloc[:,3])
start_time_so = list(df.iloc[:,4])
end_time_so = list(df.iloc[:,5])
energy_low  = list(df.iloc[:,6])
energy_high = list(df.iloc[:,7])


In [73]:
for i in range(len(uid)):
# start again at 2023-02-09T03:09
# for i in range(57,len(uid)):
    # Get the parameters for the current flare
    this_uid = uid[i]
    this_peak_time = peak_times_so[i]
    start_time = start_time_so[i][1:]
    end_time   = end_time_so[i][1:]
    e_low  = energy_low[i]
    e_high = energy_high[i]
    this_flare_loc = flare_loc_earth_string[i][1:]

    if this_uid != '' and e_low != '' and this_flare_loc != 'xxx':
        plot_aia_stix_overlay(this_uid, this_peak_time, start_time, end_time, e_low, e_high, this_flare_loc)


Found STIX map:  ['/net/galilei/work2/fschuller/data/STIX-maps/2021-03-02T23:56_2103020009_No-SAS_mem.fits']
Light travel time correction: 212.942612696 s
Export request pending. [id=JSOC_20230728_3770, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20230728_3770, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_3770, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_3770, status=1]
Waiting for 5 seconds...
1 URLs found for download. Full request totalling 12MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia.lev1_uv_24s.2021-03-03T000053Z.1700.image_lev1.fits:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Found STIX map:  ['/net/galilei/work2/fschuller/data/STIX-maps/2021-05-07T18:50_2105070034_No-SAS_mem.fits']
Light travel time correction: 45.8282952873 s
Export request pending. [id=JSOC_20230728_3774, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20230728_3774, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_3774, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_3774, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_3774, status=1]
Waiting for 5 seconds...
1 URLs found for download. Full request totalling 12MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

...

aia.lev1_uv_24s.2023-03-05T213341Z.1700.image_lev1.fits:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Found STIX map:  ['/net/galilei/work2/fschuller/data/STIX-maps/2023-03-22T23:16_2303223740_No-SAS_mem.fits']
Light travel time correction: 256.719846366 s
Export request pending. [id=JSOC_20230728_4259, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20230728_4259, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_4259, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_4259, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_4259, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20230728_4259, status=1]
Waiting for 5 seconds...
1 URLs found for download. Full request totalling 12MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

aia.lev1_uv_24s.2023-03-22T232053Z.1700.image_lev1.fits:   0%|          | 0.00/12.4M [00:00<?, ?B/s]

/home/fschuller/.local/lib/python3.11/site-packages/astropy/visualization/mpl_normalize.py:179: RuntimeWarning: invalid value encountered in divide
  np.true_divide(values, self.vmax - self.vmin, out=values)
/home/fschuller/.local/lib/python3.11/site-packages/astropy/visualization/mpl_normalize.py:179: RuntimeWarning: invalid value encountered in divide
  np.true_divide(values, self.vmax - self.vmin, out=values)


In [68]:

print(this_uid, this_peak_time, start_time, end_time, e_low, e_high, this_flare_loc)
print(i)

2103020009 2021-03-02T23:57 2021-03-02T23:56:40 2021-03-02T23:58:10 4 15 S19E43
0


# The end...

## Run one example

In [ ]:
# Define variables of interest
peak_time_so = '2022-03-28T17:07'
uid          = '2203282347'
start_time_so = '2022-03-28T17:05:10'
end_time_so  = '2022-03-28T17:07:50'
energy_low   = 18.
energy_high  = 50.

# flare location as seen from Earth
flare_loc_earth_string = 'N13W07'


In [ ]:
### Another example
peak_time_so = '2022-03-02T17:35'
uid          = '2203026385'
start_time_so = '2022-03-02T17:34:15'
end_time_so  = '2022-03-02T17:34:40'
energy_low   = 15.
energy_high  = 28.
flare_loc_earth_string = 'N15E29'


In [ ]:
i = 64
this_uid = uid[i][1:]
e_low  = energy_low[i][1:]
e_high = energy_high[i][1:]
this_flare_loc = flare_loc_earth_string[i][1:]

if this_uid != '' and e_low != '' and this_flare_loc != 'xxx':
    print ("go go")
    print (peak_times_so[i], flare_loc_earth_string[i])

    

In [ ]:
coord = flare_coord_to_float("S15W29")
p_time = '2022-03-02T17:35'
transform_HGS_to_HPC(coord, p_time)


In [ ]:
for i in range(1):
    this_uid = uid[i]
    this_peak_time = peak_times_so[i]
    start_time = start_time_so[i][1:]
    end_time   = end_time_so[i][1:]
    e_low  = energy_low[i]
    e_high = energy_high[i]
    this_flare_loc = flare_loc_earth_string[i][1:]
    print(this_uid)
    print(this_peak_time)
    print(start_time)
    print(end_time)
    print(e_low)
    print(e_high)
    print(this_flare_loc)
    

In [ ]:
plot_aia_stix_overlay(this_uid, peak_times_so[i], start_time_so[i], end_time_so[i],
                      e_low, e_high, this_flare_loc)

